In [22]:
import pymrio
import numpy as np
import pandas as pd
import os
from IPython.display import display

In [23]:
# Parse Exiobase3 (2021) data
exio3 = pymrio.parse_exiobase3(path='C:/Users/danie/Nextcloud/Coding/Masterthesis/exiobase/exio_download/IOT_2021_ixi.zip')

In [24]:
print(exio3.meta)
print(exio3)
display(exio3.get_extensions)
exio3.get_Y_categories()

Description: EXIOBASE version 3.9.5 - ixi for 2021
MRIO Name: EXIO_IOT_2021_ixi
System: ixi
Version: 3.9.5
File: None
History:
20250317 15:29:35 - MODIFICATION -  Changed country names
20250317 15:29:35 - FILEIO -  Added satellite account from land
20250317 15:29:35 - FILEIO -  Added satellite account from material
20250317 15:29:35 - FILEIO -  Added satellite account from water
20250317 15:29:34 - FILEIO -  Added satellite account from factor_inputs
20250317 15:29:34 - FILEIO -  Added satellite account from air_emissions
20250317 15:29:33 - FILEIO -  Added satellite account from employment
20250317 15:29:33 - FILEIO -  Added satellite account from nutrients
20250317 15:29:09 - FILEIO -  Loaded IO system from C:\Users\danie\Nextcloud\Coding\Masterthesis\exiobase\exio_download\IOT_2021_ixi.zip - 
IO System with parameters: Z, Y, x, A, unit, meta, nutrients, employment, air_emissions, factor_inputs, water, material, land


<bound method IOSystem.get_extensions of <pymrio.core.mriosystem.IOSystem object at 0x00000250A4476990>>

Index(['Final consumption expenditure by households',
       'Final consumption expenditure by non-profit organisations serving households (NPISH)',
       'Final consumption expenditure by government',
       'Gross fixed capital formation', 'Changes in inventories',
       'Changes in valuables', 'Exports: Total (fob)'],
      dtype='object', name='category')

In [25]:
### Check for geographical sampling differences between FIGARO and EXIOBASE 3 ###

# FIGARO countries
figaro_countries = [
    'AR', 'AT', 'AU', 'BE', 'BG', 'BR', 'CA', 'CH', 'CN', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FIGW1', 'FR', 'GB',
    'GR', 'HR', 'HU', 'ID', 'IE', 'IN', 'IT', 'JP', 'KR', 'LT', 'LU', 'LV', 'MT', 'MX', 'NL', 'NO', 'PL', 'PT', 'RO', 'RU',
    'SA', 'SE', 'SI', 'SK', 'TR', 'US', 'ZA'
]

# Extract country codes from EXIOBASE 3 dataset
exio_countries = exio3.get_regions()

# Compare country codes
common_countries = sorted(set(figaro_countries).intersection(exio_countries))
figaro_only_countries = sorted(set(figaro_countries) - set(exio_countries))
exio_only_countries = sorted(set(exio_countries) - set(figaro_countries))

print("Common countries:", common_countries)
print("Countries only in FIGARO:", figaro_only_countries)
print("Countries only in EXIOBASE 3:", exio_only_countries)
# Argentina and Saudi Arabia are not in EXIOBASE 3, but in FIGARO
# Taiwan is in EXIOBASE 3, but not in FIGARO
# FIGW1 is ROW in Figaro
# WA (Asia), WE (Europe), WF (Africa), WL (Latin America), WM (Middle East), WP (Pacific) are ROW regions in Exiobase3

# Collect all RoW regions and Taiwan in one FIGARO region
exio3.rename_regions({'WA': 'FIGW1', 'WE': 'FIGW1', 'WF': 'FIGW1', 'WL': 'FIGW1', 'WM': 'FIGW1', 'WP': 'FIGW1', 'TW': 'FIGW1'})

# Aggregate EXIOBASE 3 data to FIGARO regions
exio3.aggregate_duplicates(inplace=True)
exio3.Y.to_csv('C:/Users/danie/Nextcloud/Coding/Masterthesis/exiobase/country_agg_exio3_figaro.csv')

# Check available classification data that contains possibly useful different names and aggregation levels
mrio_class = pymrio.get_classification(mrio_name='exio3_ixi')
# Display the full mrio_class
display(mrio_class)
# Create a conversion dictionnary from ExioName to ExioLabel and check for correctness by displaying it
conv_dict = mrio_class.get_sector_dict(mrio_class.sectors.ExioName, mrio_class.sectors.ExioLabel)
display(conv_dict)

# Rename sectors in the pymrio object
exio3.rename_sectors(conv_dict)

# Check if the renaming was successful
print(exio3.Y.index)
### Aggregate Exiobase3 data ###
# Done through renaming, which also helps to adapt it to eurostat data


Common countries: ['AT', 'AU', 'BE', 'BG', 'BR', 'CA', 'CH', 'CN', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB', 'GR', 'HR', 'HU', 'ID', 'IE', 'IN', 'IT', 'JP', 'KR', 'LT', 'LU', 'LV', 'MT', 'MX', 'NL', 'NO', 'PL', 'PT', 'RO', 'RU', 'SE', 'SI', 'SK', 'TR', 'US', 'ZA']
Countries only in FIGARO: ['AR', 'FIGW1', 'SA']
Countries only in EXIOBASE 3: ['TW', 'WA', 'WE', 'WF', 'WL', 'WM']


c:\Users\danie\Nextcloud\Coding\Masterthesis\.env\Lib\site-packages\pymrio\core\mriosystem.py:2177: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  df.groupby(df.index, axis=0, sort=False)
c:\Users\danie\Nextcloud\Coding\Masterthesis\.env\Lib\site-packages\pymrio\core\mriosystem.py:2179: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  .groupby(df.columns, axis=1, sort=False)
c:\Users\danie\Nextcloud\Coding\Masterthesis\.env\Lib\site-packages\pymrio\core\mriosystem.py:2177: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  df.groupby(df.index, axis=0, sort=False)
c:\Users\danie\Nextcloud\Coding\Masterthesis\.env\Lib\site-packages\pymrio\core\mriosystem.py:2179: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  .groupby(df.columns, axis=1, sort

ClassificationData(mrio_classification_folder=WindowsPath('c:/Users/danie/Nextcloud/Coding/Masterthesis/.env/Lib/site-packages/pymrio/core/../mrio_models/exio3_ixi'), sectors_file_name='sectors.tsv', sectors_sep='\t', finaldemand_file_name='finaldemand.tsv', finaldemand_sep='\t', sectors=     ExioNumber                                           ExioName ExioCode  \
0             1                          Cultivation of paddy rice    i01.a   
1             2                               Cultivation of wheat    i01.b   
2             3                   Cultivation of cereal grains nec    i01.c   
3             4             Cultivation of vegetables, fruit, nuts    i01.d   
4             5                           Cultivation of oil seeds    i01.e   
..          ...                                                ...      ...   
158         159  Activities of membership organisation n.e.c. (91)      i91   
159         160  Recreational, cultural and sporting activities...      i92   


{'Cultivation of paddy rice': 'A_PARI',
 'Cultivation of wheat': 'A_WHEA',
 'Cultivation of cereal grains nec': 'A_OCER',
 'Cultivation of vegetables, fruit, nuts': 'A_FVEG',
 'Cultivation of oil seeds': 'A_OILS',
 'Cultivation of sugar cane, sugar beet': 'A_SUGB',
 'Cultivation of plant-based fibers': 'A_FIBR',
 'Cultivation of crops nec': 'A_OTCR',
 'Cattle farming': 'A_CATL',
 'Pigs farming': 'A_PIGS',
 'Poultry farming': 'A_PLTR',
 'Meat animals nec': 'A_OMEA',
 'Animal products nec': 'A_OANP',
 'Raw milk': 'A_MILK',
 'Wool, silk-worm cocoons': 'A_WOOL',
 'Manure treatment (conventional), storage and land application': 'A_MANC',
 'Manure treatment (biogas), storage and land application': 'A_MANB',
 'Forestry, logging and related service activities (02)': 'A_FORE',
 'Fishing, operating of fish hatcheries and fish farms; service activities incidental to fishing (05)': 'A_FISH',
 'Mining of coal and lignite; extraction of peat (10)': 'A_COAL',
 'Extraction of crude petroleum and servi

MultiIndex([('AT', 'A_PARI'),
            ('AT', 'A_WHEA'),
            ('AT', 'A_OCER'),
            ('AT', 'A_FVEG'),
            ('AT', 'A_OILS'),
            ('AT', 'A_SUGB'),
            ('AT', 'A_FIBR'),
            ('AT', 'A_OTCR'),
            ('AT', 'A_CATL'),
            ('AT', 'A_PIGS'),
            ...
            ('ZA', 'A_LANP'),
            ('ZA', 'A_LANL'),
            ('ZA', 'A_LANI'),
            ('ZA', 'A_LANT'),
            ('ZA', 'A_LANW'),
            ('ZA', 'A_ORGA'),
            ('ZA', 'A_RECR'),
            ('ZA', 'A_OSER'),
            ('ZA', 'A_PRHH'),
            ('ZA', 'A_EXTO')],
           names=['region', 'sector'], length=7172)


In [26]:

# Renaming of sectors requires mappping of ExioLabel to NACE classification

rename_dict_exio3_NACE = {
    "A_PARI": "A01",
    "A_WHEA": "A01",
    "A_OCER": "A01",
    "A_FVEG": "A01",
    "A_OILS": "A01",
    "A_SUGB": "A01",
    "A_FIBR": "A01",
    "A_OTCR": "A01",
    "A_CATL": "A01",
    "A_PIGS": "A01",
    "A_PLTR": "A01",
    "A_OMEA": "A01",
    "A_OANP": "A01",
    "A_MILK": "A01",
    "A_WOOL": "A01",
    "A_MANC": "A01",
    "A_MANB": "A01",
    "A_FORE": "A02",
    "A_FISH": "A03",
    "A_GASE": "B_gas",
    "A_OGPL": "B_gas",
    "A_COAL": "B_nongas",
    "A_COIL": "B_nongas",
    "A_ORAN": "B_nongas",
    "A_IRON": "B_nongas",
    "A_COPO": "B_nongas",
    "A_NIKO": "B_nongas",
    "A_ALUO": "B_nongas",
    "A_PREO": "B_nongas",
    "A_LZTO": "B_nongas",
    "A_ONFO": "B_nongas",
    "A_STON": "B_nongas",
    "A_SDCL": "B_nongas",
    "A_CHMF": "B_nongas",
    "A_PCAT": "C10-12",
    "A_PPIG": "C10-12",
    "A_PPLT": "C10-12",
    "A_POME": "C10-12",
    "A_VOIL": "C10-12",
    "A_DAIR": "C10-12",
    "A_RICE": "C10-12",
    "A_SUGR": "C10-12",
    "A_OFOD": "C10-12",
    "A_BEVR": "C10-12",
    "A_FSHP": "C10-12",
    "A_TOBC": "C10-12",
    "A_TEXT": "C13-15",
    "A_GARM": "C13-15",
    "A_LETH": "C13-15",
    "A_WOOD": "C16",
    "A_WOOW": "C16",
    "A_PULP": "C17",
    "A_PAPR": "C17",
    "A_PAPE": "C17",
    "A_MDIA": "C18",
    "A_COKE": "C19",
    "A_REFN": "C19",
    "A_PLAS": "C20_21",
    "A_PLAW": "C20_21",
    "A_NFER": "C20_21",
    "A_PFER": "C20_21",
    "A_CHEM": "C20_21",
    "A_RUBP": "C22",
    "A_GLAS": "C23",
    "A_GLAW": "C23",
    "A_CRMC": "C23",
    "A_BRIK": "C23",
    "A_CMNT": "C23",
    "A_ASHW": "C23",
    "A_ONMM": "C23",
    "A_NUCF": "C24",
    "A_STEL": "C24",
    "A_STEW": "C24",
    "A_PREM": "C24",
    "A_PREW": "C24",
    "A_ALUM": "C24",
    "A_ALUW": "C24",
    "A_LZTP": "C24",
    "A_LZTW": "C24",
    "A_COPP": "C24",
    "A_COPW": "C24",
    "A_ONFM": "C24",
    "A_ONFW": "C24",
    "A_METC": "C24",
    "A_FABM": "C25-33",
    "A_MACH": "C25-33",
    "A_OFMA": "C25-33",
    "A_ELMA": "C25-33",
    "A_RATV": "C25-33",
    "A_MEIN": "C25-33",
    "A_MOTO": "C25-33",
    "A_OTRE": "C25-33",
    "A_FURN": "C25-33",
    "A_POWC": "D35",
    "A_POWG": "D35",
    "A_POWN": "D35",
    "A_POWH": "D35",
    "A_POWW": "D35",
    "A_POWP": "D35",
    "A_POWB": "D35",
    "A_POWS": "D35",
    "A_POWE": "D35",
    "A_POWO": "D35",
    "A_POWM": "D35",
    "A_POWZ": "D35",
    "A_POWT": "D35",
    "A_POWD": "D35",
    "A_GASD": "D35",
    "A_HWAT": "D35",
    "A_WATR": "E36",
    "A_RYMS": "E37-39",
    "A_BOTW": "E37-39",
    "A_INCF": "E37-39",
    "A_INCP": "E37-39",
    "A_INCL": "E37-39",
    "A_INCM": "E37-39",
    "A_INCT": "E37-39",
    "A_INCW": "E37-39",
    "A_INCO": "E37-39",
    "A_BIOF": "E37-39",
    "A_BIOP": "E37-39",
    "A_BIOS": "E37-39",
    "A_COMF": "E37-39",
    "A_COMW": "E37-39",
    "A_WASF": "E37-39",
    "A_WASO": "E37-39",
    "A_LANF": "E37-39",
    "A_LANP": "E37-39",
    "A_LANL": "E37-39",
    "A_LANI": "E37-39",
    "A_LANT": "E37-39",
    "A_LANW": "E37-39",
    "A_CONS": "F",
    "A_CONW": "F",
    "A_TDMO": "G45",
    "A_TDWH": "G46",
    "A_TDFU": "G47",
    "A_TDRT": "G47",
    "A_TRAI": "H49",
    "A_TLND": "H49",
    "A_TPIP": "H49",
    "A_TWAS": "H50",
    "A_TWAI": "H50",
    "A_TAIR": "H51",
    "A_TAUX": "H52",
    "A_PTEL": "H53",
    "A_HORE": "I",
    "A_COMP": "J",
    "A_FINT": "K64",
    "A_FINS": "K65",
    "A_FAUX": "K66",
    "A_REAL": "L68",
    "A_RESD": "M_N",
    "A_OBUS": "M_N",
    "A_MARE": "M_N",
    "A_PADF": "O84",
    "A_EDUC": "P85",
    "A_HEAL": "Q",
    "A_RECR": "R_S",
    "A_ORGA": "R_S",
    "A_OSER": "R_S",
    "A_PRHH": "T",
    "A_EXTO": "U"
}


In [27]:
# Apply mapping with the rename_sectors tool of pymrio
exio3.rename_sectors(rename_dict_exio3_NACE)
print(exio3.Y.index)

# Aggregate duplicates
exio3.aggregate_duplicates()
print(exio3.Y)
exio_Y_df = exio3.Y

display(exio_Y_df)


MultiIndex([('AT',    'A01'),
            ('AT',    'A01'),
            ('AT',    'A01'),
            ('AT',    'A01'),
            ('AT',    'A01'),
            ('AT',    'A01'),
            ('AT',    'A01'),
            ('AT',    'A01'),
            ('AT',    'A01'),
            ('AT',    'A01'),
            ...
            ('ZA', 'E37-39'),
            ('ZA', 'E37-39'),
            ('ZA', 'E37-39'),
            ('ZA', 'E37-39'),
            ('ZA', 'E37-39'),
            ('ZA',    'R_S'),
            ('ZA',    'R_S'),
            ('ZA',    'R_S'),
            ('ZA',      'T'),
            ('ZA',      'U')],
           names=['region', 'sector'], length=7172)


c:\Users\danie\Nextcloud\Coding\Masterthesis\.env\Lib\site-packages\pymrio\core\mriosystem.py:2177: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  df.groupby(df.index, axis=0, sort=False)
c:\Users\danie\Nextcloud\Coding\Masterthesis\.env\Lib\site-packages\pymrio\core\mriosystem.py:2179: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  .groupby(df.columns, axis=1, sort=False)
c:\Users\danie\Nextcloud\Coding\Masterthesis\.env\Lib\site-packages\pymrio\core\mriosystem.py:2177: FutureWarning: The 'axis' keyword in DataFrame.groupby is deprecated and will be removed in a future version.
  df.groupby(df.index, axis=0, sort=False)
c:\Users\danie\Nextcloud\Coding\Masterthesis\.env\Lib\site-packages\pymrio\core\mriosystem.py:2179: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  .groupby(df.columns, axis=1, sort

region                                                   AT  \
category        Final consumption expenditure by households   
region sector                                                 
AT     A01                                       631.427683   
       A02                                       291.844474   
       A03                                        42.905383   
       B_nongas                                  133.993108   
       B_gas                                      18.004569   
...                                                     ...   
ZA     P85                                         0.081434   
       Q                                           1.414897   
       R_S                                         0.190255   
       T                                           6.487142   
       U                                           0.000000   

region                                                                                                \
category     

region                                                   AT  \
category        Final consumption expenditure by households   
region sector                                                 
AT     A01                                       631.427683   
       A02                                       291.844474   
       A03                                        42.905383   
       B_nongas                                  133.993108   
       B_gas                                      18.004569   
...                                                     ...   
ZA     P85                                         0.081434   
       Q                                           1.414897   
       R_S                                         0.190255   
       T                                           6.487142   
       U                                           0.000000   

region                                                                                                \
category        Final consumption expenditure by non-profit organisations serving households (NPISH)   
region sector                                                                                          
AT     A01                                                0.000489                                     
       A02                                                0.000000                                     
       A03                                                0.000000                                     
       B_nongas                                           0.005511                                     
       B_gas                                              0.000081                                     
...                                                            ...                                     
ZA     P85                                                0.023644                                     
       Q                                                  0.199135                                     
       R_S                                                0.089581                                     
       T                                                  0.000000                                     
       U                                                  0.000000                                     

region                                                       \
category        Final consumption expenditure by government   
region sector                                                 
AT     A01                                     2.631293e+02   
       A02                                     4.686303e-06   
       A03                                     0.000000e+00   
       B_nongas                                8.546216e+00   
       B_gas                                   2.924536e+00   
...                                                     ...   
ZA     P85                                     5.794555e-01   
       Q                                       9.006513e-07   
       R_S                                     1.876217e-01   
       T                                       0.000000e+00   
       U                                       0.000000e+00   

region                                                                \
category        Gross fixed capital formation Changes in inventories   
region sector                                                          
AT     A01                         501.298815           5.601037e+02   
       A02                           0.159544           2.173132e+02   
       A03                           0.000000           2.841954e+00   
       B_nongas                     80.343494           1.046078e+02   
       B_gas                         0.764447           2.018026e+00   
...                                       ...                    ...   
ZA     P85                           0.000290           8.652842e-05   
       Q                             0.000001           2

In [33]:
# Extract energy inputs for B_gas and B_nongas (rows)
energy_inputs = exio_Y_df.loc[(slice(None), ['B_gas', 'B_nongas']), :]

# Compute total energy input per row (sum of B_gas and B_nongas)
total_energy_inputs_row = energy_inputs.groupby(level=0).sum()
total_energy_inputs_row.replace(0, np.nan, inplace=True)  # Avoid division by zero

# Compute row-wise shares (only for B_gas and B_nongas rows)
energy_shares_row = energy_inputs.div(total_energy_inputs_row, level=0).fillna(0)

# Create a copy of the original DataFrame to preserve structure
exio_Y_with_row_shares = exio_Y_df.copy()

# Add energy share values to the original dataframe (only updating B_gas and B_nongas rows)
exio_Y_with_row_shares.loc[energy_shares_row.index] = energy_shares_row

# Display the full table with updated B_gas and B_nongas rows
display(exio_Y_with_row_shares)

region                                                   AT  \
category        Final consumption expenditure by households   
region sector                                                 
AT     A01                                       631.427683   
       A02                                       291.844474   
       A03                                        42.905383   
       B_nongas                                    0.881547   
       B_gas                                       0.118453   
...                                                     ...   
ZA     P85                                         0.081434   
       Q                                           1.414897   
       R_S                                         0.190255   
       T                                           6.487142   
       U                                           0.000000   

region                                                                                                \
category        Final consumption expenditure by non-profit organisations serving households (NPISH)   
region sector                                                                                          
AT     A01                                                0.000489                                     
       A02                                                0.000000                                     
       A03                                                0.000000                                     
       B_nongas                                           0.985569                                     
       B_gas                                              0.014431                                     
...                                                            ...                                     
ZA     P85                                                0.023644                                     
       Q                                                  0.199135                                     
       R_S                                                0.089581                                     
       T                                                  0.000000                                     
       U                                                  0.000000                                     

region                                                       \
category        Final consumption expenditure by government   
region sector                                                 
AT     A01                                     2.631293e+02   
       A02                                     4.686303e-06   
       A03                                     0.000000e+00   
       B_nongas                                7.450441e-01   
       B_gas                                   2.549559e-01   
...                                                     ...   
ZA     P85                                     5.794555e-01   
       Q                                       9.006513e-07   
       R_S                                     1.876217e-01   
       T                                       0.000000e+00   
       U                                       0.000000e+00   

region                                                                \
category        Gross fixed capital formation Changes in inventories   
region sector                                                          
AT     A01                         501.298815           5.601037e+02   
       A02                           0.159544           2.173132e+02   
       A03                           0.000000           2.841954e+00   
       B_nongas                      0.990575           9.810738e-01   
       B_gas                         0.009425           1.892623e-02   
...                                       ...                    ...   
ZA     P85                           0.000290           8.652842e-05   
       Q                             0.000001           2